In [ ]:
#!pip3 install -qU neptune

In [ ]:
import torch
import random
import numpy as np


SEED = 1000

def setup_reproducibility():
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    torch.set_float32_matmul_precision("high")

setup_reproducibility()

In [ ]:
def return_stats(tensor, p=True, r=False):
    mean, std = tensor.mean(), tensor.std()
    min, max =  tensor.min(), tensor.max()
    
    if p: print(f"Min: {min}, Max: {max}, Mean: {mean}, Std: {std}")
    
    if r: return min, max, mean, std
    
    
def zscore(tensor, mean=None, std=None):
    if mean is None: mean = tensor.mean()
    if std is None: std = tensor.std()
    return (tensor - mean) / (std + 1e-6)


def get_model_size(model):
    print(sum(p.numel() for p in model.parameters()) / 1e6)

In [ ]:
import os

path = "/kaggle/input/dig-4-bio-raman-transfer-learning-challenge"
files = os.listdir(path)
[(i, files[i]) for i in range(len(files))]

In [ ]:
train_inputs = []
train_targets = []

In [ ]:
import pandas as pd

timegate = pd.read_csv(os.path.join(path, files[1]))

timegate.drop(columns="fold_idx", inplace=True)
timegate.drop(columns="MSM_present", inplace=True)
timegate_inputs = timegate[timegate.columns[:-3]].to_numpy()
timegate_targets = timegate[timegate.columns[-3:]].to_numpy()

train_inputs.append(timegate_inputs)
train_targets.append(timegate_targets)

In [ ]:
mettler_toledo = pd.read_csv(os.path.join(path, files[2]))

mettler_toledo.drop(columns="fold_idx", inplace=True)
mettler_toledo.drop(columns="MSM_present", inplace=True)
mettler_toledo_inputs = mettler_toledo[mettler_toledo.columns[:-3]].to_numpy()
mettler_toledo_targets = mettler_toledo[mettler_toledo.columns[-3:]].to_numpy()

train_inputs.append(mettler_toledo_inputs)
train_targets.append(mettler_toledo_targets)

In [ ]:
kaiser = pd.read_csv(os.path.join(path, files[3]))

kaiser.drop(columns="fold_idx", inplace=True)
kaiser.drop(columns="MSM_present", inplace=True)
kaiser_inputs = kaiser[kaiser.columns[:-3]].to_numpy()
kaiser_targets = kaiser[kaiser.columns[-3:]].to_numpy()

train_inputs.append(kaiser_inputs)
train_targets.append(kaiser_targets)

In [ ]:
anton = pd.read_csv(os.path.join(path, files[4]))

anton.drop(columns="fold_idx", inplace=True)
anton.drop(columns="MSM_present", inplace=True)
anton_inputs = anton[anton.columns[:-3]].to_numpy()
anton_targets = anton[anton.columns[-3:]].to_numpy()

train_inputs.append(anton_inputs)
train_targets.append(anton_targets)

In [ ]:
tornado = pd.read_csv(os.path.join(path, files[7]))

tornado.drop(columns="fold_idx", inplace=True)
tornado.drop(columns="MSM_present", inplace=True)
tornado_inputs = tornado[tornado.columns[:-3]].to_numpy()
tornado_targets = tornado[tornado.columns[-3:]].to_numpy()

train_inputs.append(tornado_inputs)
train_targets.append(tornado_targets)

In [ ]:
csv_path = os.path.join(path, files[8])
print(csv_path)
tec5 = pd.read_csv(csv_path)

tec5.drop(columns="fold_idx", inplace=True)
tec5.drop(columns="MSM_present", inplace=True)
tec5_inputs = tec5[tec5.columns[:-3]].to_numpy()
tec5_targets = tec5[tec5.columns[-3:]].to_numpy()

train_inputs.append(tec5_inputs)
train_targets.append(tec5_targets)

In [ ]:
csv_path = os.path.join(path, files[9])
print(csv_path)
metrohm = pd.read_csv(csv_path)

metrohm.drop(columns="fold_idx", inplace=True)
metrohm.drop(columns="MSM_present", inplace=True)
metrohm_inputs = metrohm[metrohm.columns[:-3]].to_numpy()
metrohm_targets = metrohm[metrohm.columns[-3:]].to_numpy()

train_inputs.append(metrohm_inputs)
train_targets.append(metrohm_targets)

In [ ]:
csv_path = os.path.join(path, files[10])
print(csv_path)
anton785 = pd.read_csv(csv_path)

anton785.drop(columns="fold_idx", inplace=True)
anton785.drop(columns="MSM_present", inplace=True)
anton785_inputs = anton785[anton785.columns[:-3]].to_numpy()
anton785_targets = anton785[anton785.columns[-3:]].to_numpy()

train_inputs.append(anton785_inputs)
train_targets.append(anton785_targets)

In [ ]:
import torch
import torch.nn.functional as F


for i in range(len(train_inputs)):
    x = train_inputs[i]
    x = torch.tensor(x).unsqueeze(0)
    x = F.interpolate(x, size=2048, mode="nearest-exact")
    train_inputs[i] = x.squeeze()
    
train_inputs = torch.cat(train_inputs)
train_targets = [torch.tensor(t) for t in train_targets]
train_targets = torch.cat(train_targets)

train_inputs.shape, train_targets.shape

In [ ]:
low = torch.quantile(train_inputs, 0.02, interpolation="linear")
high = torch.quantile(train_inputs, 0.98, interpolation="linear")
train_inputs = train_inputs.clamp(low, high)

In [ ]:
_, _, mean, std = return_stats(train_inputs, r=True)
train_inputs = zscore(train_inputs, mean, std)
return_stats(train_inputs)

In [ ]:
csv_path = os.path.join(path, files[5])
print(csv_path)
eval_set = pd.read_csv(csv_path)

eval_input_cols = eval_set.columns[1:2049]
eval_target_cols = eval_set.columns[2050:]

In [ ]:
eval_targets  = eval_set[eval_target_cols].dropna().to_numpy()
eval_targets[:5]

In [ ]:
eval_set = eval_set[eval_input_cols]
eval_set['Unnamed: 1'] = eval_set['Unnamed: 1'].str.replace('[\[\]]', '', regex=True).astype('int64')
eval_set['Unnamed: 2048'] = eval_set['Unnamed: 2048'].str.replace('[\[\]]', '', regex=True).astype('int64')

In [ ]:
eval_inputs = eval_set.to_numpy().reshape(-1, 2, 2048)
eval_inputs = eval_inputs.mean(axis=1)

In [ ]:
eval_inputs = torch.tensor(eval_inputs)
eval_targets = torch.tensor(eval_targets)

In [ ]:
return_stats(eval_inputs)
eval_inputs = eval_inputs.clamp(low, high)
return_stats(eval_inputs)
eval_inputs = zscore(eval_inputs, mean, std)
return_stats(eval_inputs)

In [ ]:
from sklearn.model_selection import train_test_split


def split():
        train_inputs, eval_inputs, train_targets, eval_targets = train_test_split(
                inputs,
                targets,                      
                test_size=0.2,
                random_state=1000,
                shuffle=True
        )

        train_inputs.shape, eval_inputs.shape, train_targets.shape, eval_targets.shape

In [ ]:
train_inputs = train_inputs.float().unsqueeze(1)
train_targets = train_targets.float()
eval_inputs = eval_inputs.float().unsqueeze(1)
eval_targets = eval_targets.float()

In [ ]:
train_inputs.shape

In [ ]:
from torch.utils.data import TensorDataset

train_ds = TensorDataset(train_inputs, train_targets)
eval_ds = TensorDataset(eval_inputs, eval_targets)

In [ ]:
from torch.utils.data import DataLoader


def build_loader(
    SEED,
    ds,
    train=True,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
):
    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    generator = torch.Generator()
    generator.manual_seed(SEED if train else SEED+1)

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers,
        worker_init_fn=seed_worker,
        generator=generator,
        #sampler=DistributedSampler(
        #    train_ds,
        #    shuffle=True,
        #    drop_last=True,
        #    seed=config.seed
        #)
    )
    
    
def return_dls(train_ds, eval_ds):
    train_dl = build_loader(
        SEED,
        train_ds,
        train=True,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )

    eval_dl = build_loader(
        SEED,
        eval_ds,
        train=False,
        batch_size=96,
        shuffle=False,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )
    
    return train_dl, eval_dl

In [ ]:
"""
resnet for 1-d signal data, pytorch version
Shenda Hong, Oct 2019
https://github.com/hsd1503/resnet1d

Modified by Hiroshi Yoshihara
"""
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyConv1dPadSame(nn.Module):
    """
    extend nn.Conv1d to support SAME padding
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1):
        super(MyConv1dPadSame, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.conv = torch.nn.Conv1d(
            in_channels=self.in_channels, 
            out_channels=self.out_channels, 
            kernel_size=self.kernel_size, 
            stride=self.stride, 
            groups=self.groups)

    def forward(self, x):
        
        net = x
        
        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)
        
        net = self.conv(net)

        return net

      
class MyMaxPool1dPadSame(nn.Module):
    """
    extend nn.MaxPool1d to support SAME padding
    """
    def __init__(self, kernel_size):
        super(MyMaxPool1dPadSame, self).__init__()
        self.kernel_size = kernel_size
        self.stride = 1
        self.max_pool = torch.nn.MaxPool1d(kernel_size=self.kernel_size)

    def forward(self, x):
        
        net = x
        
        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)
        
        net = self.max_pool(net)
        
        return net


class BasicBlock(nn.Module):
    """
    ResNet Basic Block
    """
    def __init__(self, 
                 in_channels, 
                 out_channels, 
                 kernel_size, 
                 stride, 
                 groups, 
                 downsample, 
                 use_bn, 
                 dropout=0.2, 
                 activation=nn.ReLU,
                 is_first_block=False):
        super(BasicBlock, self).__init__()
        
        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.stride = stride
        self.groups = groups
        self.downsample = downsample
        if self.downsample:
            self.stride = stride
        else:
            self.stride = 1
        self.is_first_block = is_first_block
        self.use_bn = use_bn
        self.dropout = dropout
        self.activation = activation

        # the first conv
        self.bn1 = nn.BatchNorm1d(in_channels)
        self.relu1 = self.activation(inplace=True)
        self.do1 = nn.Dropout(p=self.dropout)
        self.conv1 = MyConv1dPadSame(
            in_channels=in_channels, 
            out_channels=out_channels, 
            kernel_size=kernel_size, 
            stride=self.stride,
            groups=self.groups)

        # the second conv
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = self.activation(inplace=True)
        self.do2 = nn.Dropout(p=self.dropout)
        self.conv2 = MyConv1dPadSame(
            in_channels=out_channels, 
            out_channels=out_channels, 
            kernel_size=kernel_size, 
            stride=1,
            groups=self.groups)
                
        self.max_pool = MyMaxPool1dPadSame(kernel_size=self.stride)

    def forward(self, x):
        
        identity = x
        
        # the first conv
        out = x
        if not self.is_first_block:
            if self.use_bn:
                out = self.bn1(out)
            out = self.relu1(out)
            out = self.do1(out)
        out = self.conv1(out)
        
        # the second conv
        if self.use_bn:
            out = self.bn2(out)
        out = self.relu2(out)
        out = self.do2(out)
        out = self.conv2(out)
        
        # if downsample, also downsample identity
        if self.downsample:
            identity = self.max_pool(identity)
            
        # if expand channel, also pad zeros to identity
        if self.out_channels != self.in_channels:
            identity = identity.transpose(-1,-2)
            ch1 = (self.out_channels-self.in_channels)//2
            ch2 = self.out_channels-self.in_channels-ch1
            identity = F.pad(identity, (ch1, ch2), "constant", 0)
            identity = identity.transpose(-1,-2)
        
        # shortcut
        out += identity

        return out


class ResNet1d(nn.Module):
    """
    
    Input:
        X: (n_samples, n_channel, n_length)
        Y: (n_samples)
        
    Output:
        out: (n_samples)
        
    Pararmetes:
        in_channels: dim of input, the same as n_channel
        base_filters: number of filters in the first several Conv layer, it will double at every 4 layers
        kernel_size: width of kernel
        stride: stride of kernel moving
        groups: set larget to 1 as ResNeXt
        n_block: number of blocks
        n_classes: number of classes
        
    """

    def __init__(self, 
                 in_channels, 
                 base_filters, 
                 kernel_size, 
                 stride=2, 
                 groups=32, 
                 n_block=16, 
                 n_classes=1, 
                 downsample_gap=None, 
                 increasefilter_gap=None, 
                 use_bn=True, 
                 dropout=0.2, 
                 activation=nn.ReLU,
                 final_act=True,
                 reinit=False):
        super().__init__()
        
        self.n_block = n_block
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.use_bn = use_bn
        self.dropout = dropout
        self.activation = activation
        self.final_act = final_act

        if downsample_gap is None:
            downsample_gap = self.n_block // 8
        self.downsample_gap = downsample_gap # 2 for base model
        if increasefilter_gap is None:
            increasefilter_gap = self.n_block // 4
        self.increasefilter_gap = increasefilter_gap # 4 for base model

        # first block
        self.first_block_conv = MyConv1dPadSame(
            in_channels=in_channels, 
            out_channels=base_filters, 
            kernel_size=self.kernel_size, 
            stride=1)
        self.first_block_bn = nn.BatchNorm1d(base_filters)
        self.first_block_relu = self.activation(inplace=True)
        out_channels = base_filters
                
        # residual blocks
        self.basicblock_list = nn.ModuleList()
        for i_block in range(self.n_block):
            # is_first_block
            if i_block == 0:
                is_first_block = True
            else:
                is_first_block = False
            # downsample at every self.downsample_gap blocks
            if i_block % self.downsample_gap == 1:
                downsample = True
            else:
                downsample = False
            # in_channels and out_channels
            if is_first_block:
                in_channels = base_filters
                out_channels = in_channels
            else:
                # increase filters at every self.increasefilter_gap blocks
                in_channels = int(base_filters*2**((i_block-1)//self.increasefilter_gap))
                if (i_block % self.increasefilter_gap == 0) and (i_block != 0):
                    out_channels = in_channels * 2
                else:
                    out_channels = in_channels
            
            tmp_block = BasicBlock(
                in_channels=in_channels, 
                out_channels=out_channels, 
                kernel_size=self.kernel_size, 
                stride = self.stride, 
                groups = self.groups, 
                downsample=downsample, 
                use_bn = self.use_bn,  
                dropout = self.dropout, 
                activation = self.activation,
                is_first_block=is_first_block)
            self.basicblock_list.append(tmp_block)

        # final prediction
        self.final_bn = nn.BatchNorm1d(out_channels)
        self.final_relu = self.activation(inplace=True)
        self.final_pool = nn.AdaptiveAvgPool1d(1)
        self.dense = nn.Linear(out_channels, n_classes)

        # init 
        if reinit:
            for m in self.modules():
                if isinstance(m, nn.Conv1d):
                    nn.init.kaiming_normal_(m.weight)
                elif isinstance(m, nn.BatchNorm1d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        out = self.first_block_conv(x)
        if self.use_bn:
            out = self.first_block_bn(out)
        out = self.first_block_relu(out)
        
        # residual blocks, every block has two conv
        for i_block in range(self.n_block):
            net = self.basicblock_list[i_block]
            out = net(out)
        
        # final prediction
        if self.final_act:
            if self.use_bn:
                out = self.final_bn(out)
            out = self.final_relu(out)

        # out = out.mean(-1)
        out = self.final_pool(out)
        return out

    def forward(self, x):
        out = self.forward_features(x)
        out = out.squeeze(-1)
        out = self.dense(out)
        return out    

    def reset_classifier(self, **kwargs):
        self.dense = nn.Identity()

    def get_classifier(self):
        return self.dense


In [ ]:
import neptune


def setup_neptune():
    if not RESUME:
        neptune_run = neptune.init_run(
            project="arbaaz/kaggle-spect",
            name=MODEL_NAME,
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
        )

        neptune_run["h_parameters"] = {
            "seed": SEED,
            "model_name": MODEL_NAME,
            "optimizer_name": "nadam",
            "learning_rate": LR,
            "scheduler_name": "default",
            "weight_decay": WD,
            "dropout": DROPOUT,
            "num_epochs": EPOCHS,
            "batch_size": BATCH_SIZE,
        }
    else:
        neptune_run = neptune.init_run(
            project="arbaaz/crunchdao-structural-break",
            with_id=config.with_id,
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
        )

    return neptune_run

In [ ]:
import torch.nn.functional as F
from sklearn.metrics import r2_score


def loss_fn(logits, targets):
    logits = logits.view(-1)
    targets = targets.view(-1)
    return F.mse_loss(logits, targets)


def metric_fn(logits, targets):
    preds = logits.cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    one = r2_score(targets[:, 0], preds[:, 0])
    two = r2_score(targets[:, 1], preds[:, 1])
    three = r2_score(targets[:, 2], preds[:, 2])
    mean_r2 = (one + two + three) / 3
    return one, two, three, mean_r2

In [ ]:
import torch.nn as nn

class Model(nn.Module):
    """
    MLP optimized for Raman spectroscopy concentration prediction
    """
    def __init__(self, input_size=train_inputs.shape[1], hidden_sizes=[128], 
                 output_size=3, dropout_rate=0.3, use_batch_norm=True):
        super().__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        self.use_batch_norm = use_batch_norm
        
        # Build the network layers
        layers = []
        prev_size = input_size
        
        for i, hidden_size in enumerate(hidden_sizes):
            # Linear layer
            layers.append(nn.Linear(prev_size, hidden_size))
            
            # Batch normalization
            if use_batch_norm:
                layers.append(nn.BatchNorm1d(hidden_size))
            
            # Activation
            layers.append(nn.GELU())
            
            # Dropout (not on the last hidden layer)
            if i < len(hidden_sizes) - 1:
                layers.append(nn.Dropout(dropout_rate))
            
            prev_size = hidden_size
        
        # Output layer
        layers.append(nn.Linear(prev_size, output_size))
        
        self.network = nn.Sequential(*layers)
        
        # Initialize weights
        self._initialize_weights()
        self.d = nn.Dropout(dropout_rate)
    
    def _initialize_weights(self):
        """Initialize network weights using Xavier/Glorot initialization"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_normal_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
    
    def forward(self, x):
        x = self.d(x)
        return self.network(x)


In [ ]:
import random
import warnings
from transformers import get_cosine_schedule_with_warmup
from tqdm.auto import tqdm

#warnings.filterwarnings("ignore")

MODEL_NAME = "ResNet.Kernel3"
EPOCHS = 1000
BATCH_SIZE = 17
WD = 1e-4
LR = 1e-4
DROPOUT = 0.3
SCORE = float('-inf')
LOG = True
RESUME = False
device = "cuda" if torch.cuda.is_available() else "cpu"



#model = Model(hidden_sizes=[512, 512, 512, 512, 512, 512], dropout_rate=DROPOUT).to(device)
model = ResNet1d(1, base_filters=64, kernel_size=3, n_classes=3).to(device)
get_model_size(model)
#model = nn.DataParallel(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD, foreach=True)
scaler = torch.amp.GradScaler(device)
train_dl, eval_dl = return_dls(train_ds, eval_ds)

total_training_steps = len(train_dl) * EPOCHS
warmup_steps = int(total_training_steps * 0.05)  # e.g. 5% warmup
scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_training_steps
)

if LOG:
    neptune_run = setup_neptune()



for epoch in tqdm(range(EPOCHS)):
    model.train()
    total_loss = 0.0
    all_logits = []
    all_targets = []
    
    for inputs, targets in train_dl:
        inputs = inputs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        
        #if random.random() < 0.7:
        #    noise = torch.randn_like(inputs) * 0.1  # std=0.1
        #    inputs = inputs + noise

        with torch.amp.autocast(device_type=device, dtype=torch.float16, cache_enabled=True):
            logits = model(inputs)
            loss = loss_fn(logits, targets)
        
        if LOG:  neptune_run["lr_step"].append(scheduler.get_last_lr()[0])
        scheduler.step()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        
        total_loss += loss.detach().cpu()
        all_logits.append(logits.detach().cpu())
        all_targets.append(targets.detach().cpu())
    
    all_logits = torch.cat(all_logits)
    all_targets = torch.cat(all_targets)

    one, two, three, r2 = metric_fn(all_logits, all_targets)
    total_loss = total_loss / len(train_dl)
    
    
    model.eval()
    eval_total_loss = 0.0
    eval_all_logits = []
    eval_all_targets = []

    for inputs, targets in eval_dl:
        inputs = inputs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        with torch.inference_mode():
            with torch.amp.autocast(device_type=device, dtype=torch.float16, cache_enabled=True):
                logits = model(inputs)
                loss = loss_fn(logits, targets)

        eval_total_loss += loss.detach().cpu()
        eval_all_logits.append(logits.detach().cpu())
        eval_all_targets.append(targets.detach().cpu())
    
    eval_all_logits = torch.cat(eval_all_logits)
    eval_all_targets = torch.cat(eval_all_targets)

    eval_one, eval_two, eval_three, eval_r2 = metric_fn(eval_all_logits, eval_all_targets)
    eval_total_loss = eval_total_loss / len(eval_dl)
    
    if eval_r2 > SCORE:
        SCORE = eval_r2
        data = {"state_dict": model.state_dict()}
        data["epoch"] = epoch 
        data["score"] = SCORE
        torch.save(data, "/kaggle/working/ckpt.pt")
    
    if LOG:
        neptune_run["train/loss"].append(total_loss)
        neptune_run["eval/loss"].append(eval_total_loss)
        neptune_run["train/r2"].append(r2)
        neptune_run["eval/r2"].append(eval_r2)
        neptune_run["train/one"].append(one)
        neptune_run["train/two"].append(two)
        neptune_run["train/three"].append(three)
        neptune_run["eval/one"].append(eval_one)
        neptune_run["eval/two"].append(eval_two)
        neptune_run["eval/three"].append(eval_three)
        
    if True:
        print(
            f"Epoch: {epoch}, "
            f"train/loss: {total_loss:.4f}, "
            f"eval/loss: {eval_total_loss:.4f}, "
            f"train/r2: {r2:.4f}, "
            f"eval/r2: {eval_r2:.4f}, "
            f"train/one: {one:.4f}, "
            f"train/two: {two:.4f}, "
            f"train/three: {three:.4f}, "
            f"eval/one: {eval_one:.4f}, "
            f"eval/two: {eval_two:.4f}, "
            f"eval/three: {eval_three:.4f} "
        )
        
if LOG:
    neptune_run.stop()

In [ ]:
import torch
ckpt_path = "/kaggle/working/ckpt.pt"
ckpt = torch.load(ckpt_path, weights_only=False)
ckpt["epoch"], ckpt["score"]

In [ ]:

#def load_test():    
test = pd.read_csv(os.path.join(path, files[6]))

row1 = test.columns[1:].to_numpy().copy()
row1[-1] = "5611"
row1 = row1.astype(np.float64)


cols = test.columns[1:]
test = test[cols]
test[" 5611]"] = test[" 5611]"].str.replace('[\[\]]', '', regex=True).astype('int64')
test = test.to_numpy()

test = np.insert(test, 0, row1, axis=0)
test = test.reshape(-1, 2, 2048).mean(axis=1)

#test_features = []
#for i in test:
#    f = extract_features(i)
#    
#    values = []
#    for v in f.values():
#        values.append(v)
# 
#    values = np.stack(values).ravel()
 #   test_features.append(values)

#test_features = np.stack(test_features)
#print(test_features.shape)

#test = test_features
return_stats(test)
test = torch.tensor(test)
test = test.clamp(low, high)
return_stats(test)
test = zscore(test, mean, std).float()
test.shape, test.dtype, return_stats(test)

In [ ]:
test = test.unsqueeze(1)
test.shape

In [ ]:
device = "cuda"
model = ResNet1d(1, base_filters=64, kernel_size=3, n_classes=3).to(device)
model.load_state_dict(ckpt["state_dict"])
model.eval()

with torch.inference_mode():
    preds = model(test.cuda())

preds = preds.cpu().detach().numpy()
preds.shape

In [ ]:
#def prepare_test():
column_names = ['Glucose', 'Sodium Acetate', 'Magnesium Sulfate']
preds_df = pd.DataFrame(preds, columns=column_names)
preds_df.insert(0, 'ID', [i+1 for i in range(len(preds_df))])
preds_df

In [ ]:
#def save_test():
name = "ResNet.-0.1588.csv"
preds_df.to_csv(name, index=False)
f = pd.read_csv(f"/kaggle/working/{name}")
f